In [2]:
from qutip import *
import matplotlib.pyplot as plt
import numpy as np
from helper_code_qutip import *
from scipy import integrate
import scipy.io

In [5]:
Tc_list = [0.001,0.18,0.20]
Th_list = [0.1] #For some reason, integral diverges outside this range


betalist1 = [1/Tc for Tc in Tc_list]
betalist2 = [1/Th for Th in Th_list]

gvals = np.linspace(0.001,1,60)

print(len(betalist1),len(betalist2))

redfield_ss = []
ele_ss = []
lle_ss = []
lle_ss2 = []
e = 0.01
for beta2 in betalist2:
    list_red = []
    list_lle = []
    list_lle2 = []
    list_ele = []
    for g in gvals:
        limit_value=700
        b=500
        N=2
        w0min=1
        w0max=w0min+e
        delta=1
        gmin=g
        gmax=g
        beta1 = 1/0.1  #Tc = 0.1
            
        w0list=np.linspace(w0min,w0max,N)
        glist=np.linspace(gmin,gmax,N-1)
            
        tb=0.01
        epsilon=1
        gamma1=1e-3 #gamma1 is the coupling to left bath. It shows up in spectral bath function
        gamma2=1.1e-2   #gamma2 is the coupling to the right bath.    
            
            

        mu=0
            
            
        delta=1
        mu1=mu
        mu2=mu

        H_S=create_hamiltonian2(w0list,glist,N)
        rho_th = (-beta2*H_S).expm()/((-beta2*H_S).expm()).tr()
        rho_th1 = to_super(rho_th)
        rho_th2 = operator_to_vector(rho_th)
    
    
        c_N=create_sm(N,N)  # we couple the Nth spin to the bath
        c_1=create_sm(N,1)
        
        
        
        eigenergies,eigstates=H_S.eigenstates()
        
        #print("eigenenergies are : ",eigenergies)
        
        spectrum=max(eigenergies)-min(eigenergies)
        
        
        
        number=len(eigenergies)
        
        integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
        integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
        integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
        integral22=np.empty((number,number),dtype=np.cdouble)

        print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

        for i in range(number):
            for k in range(number):
                freq=eigenergies[k]-eigenergies[i]
                print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                #print(i,k,freq)
                if( np.absolute(freq) >= 1/10**10):
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                    integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath
        
                if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                    integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
                    integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma1),limit=limit_value)[0]
                    integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                    integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma2),limit=limit_value)[0]
                
            
            #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
        #        print(i,k,integral2[i,k],expected)
    
    
        # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
    
        constant12=np.empty((number,number),dtype=np.cdouble)
        constant11=np.empty((number,number),dtype=np.cdouble)
        constant21=np.empty((number,number),dtype=np.cdouble)
        constant22=np.empty((number,number),dtype=np.cdouble)
        
        
        
        for i in range(number):
            for k in range(number):
                constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1))    #full coefficient created this is nbar+1
                constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1)                                       # the full coefficient is created
                
                constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created
                #print(i,k,constant11[i,k],constant12[i,k],constant21[i,k],constant22[i,k])
        list1=[]
        list2=[]
        
        
        for i in range(number):
            list1.append([])
            list2.append([])
        
        
        
        matrix=np.zeros((number,number))
        
        dim=[]
        for k in range(N):
            dim.append(2)    
        
        zeromatrix=Qobj(matrix,dims=[dim,dim])
        
        
        indices1=[]
        indices2=[]
        
        
        
        
        for i in range(number):
            for k in range(number):
                list1[i].append(eigstates[i]*eigstates[i].dag()*c_1*eigstates[k]*eigstates[k].dag())
                list2[i].append(eigstates[i]*eigstates[i].dag()*c_N*eigstates[k]*eigstates[k].dag())
                
                if(tracedist(eigstates[i]*eigstates[i].dag()*c_1*eigstates[k]*eigstates[k].dag(),zeromatrix)!=0):
                    indices1.append((i,k))
                if(tracedist(eigstates[i]*eigstates[i].dag()*c_N*eigstates[k]*eigstates[k].dag(),zeromatrix)!=0):
                    indices2.append((i,k))

        pre=-1.0j*H_S
        post=1.0j*H_S
        
        L=spre(pre)+spost(post)
        
        for i in range(number):
            for k in range(number):
                print(constant11[i,k],constant12[i,k])
                for j in range(number):
                    for m in range(number):
                        vi=eigstates[i]
                        vk=eigstates[k]

                        vj=eigstates[j]
                        vm=eigstates[m]

                        #For ELE, we only take terms where vm - vj = vk - vi

                        

                        if vm - vj == vk - vi:
                        
                            op1=epsilon*epsilon*constant11[i,k]*vi*vi.dag()*c_1*vk*vk.dag()*vm*vm.dag()*c_1.dag()*vj*vj.dag()
                            op2=epsilon*epsilon*constant12[i,k]*vm*vm.dag()*c_1.dag()*vj*vj.dag()*vi*vi.dag()*c_1*vk*vk.dag()
                            
                            op3=epsilon*epsilon*constant11[i,k]*vm*vm.dag()*c_1.dag()*vj*vj.dag()
                            op4=vi*vi.dag()*c_1*vk*vk.dag()
                            op5=epsilon*epsilon*constant12[i,k]*vm*vm.dag()*c_1.dag()*vj*vj.dag()
                            
                            
                            L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
                            L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                            
                            op1=epsilon*epsilon*constant21[i,k]*vi*vi.dag()*c_N*vk*vk.dag()*vm*vm.dag()*c_N.dag()*vj*vj.dag()
                            op2=epsilon*epsilon*constant22[i,k]*vm*vm.dag()*c_N.dag()*vj*vj.dag()*vi*vi.dag()*c_N*vk*vk.dag()
                            
                            op3=epsilon*epsilon*constant21[i,k]*vm*vm.dag()*c_N.dag()*vj*vj.dag()
                            op4=vi*vi.dag()*c_N*vk*vk.dag()
                            op5=epsilon*epsilon*constant22[i,k]*vm*vm.dag()*c_N.dag()*vj*vj.dag()
                            
                            
                            L=L+spre(-op2-op1.dag())+spost(-op1-op2.dag())
                            L=L+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                        
                        
                        
                #Variables needed for for iterative-lgmres to work. 
                return_info=True
                rho_fin1 = L*rho_th1
                rho_fin2 = L*rho_th2

                print(rho_fin1)
                print(rho_fin2)
                #print('Redfield Liouvillian constructed, Computing steady-state ...')
                """ss_ele = steadystate(L,return_info=return_info)
                L_eigen = L.eigenenergies()
                print("Smallest eigenvalues of L_ele are ", L_eigen[-3:])
                list_ele.append(ss_ele)

################################## Redfield stuff ########################################

        pre=-1.0j*H_S
        post=1.0j*H_S
        
        L_red=spre(pre)+spost(post)
        
        for i in range(number):
            for k in range(number):
                vi=eigstates[i]
                vk=eigstates[k]

                print(constant11[i,k],constant12[i,k])
                
                op1=epsilon*epsilon*constant11[i,k]*vi*vi.dag()*c_1*vk*vk.dag()*c_1.dag()
                op2=epsilon*epsilon*constant12[i,k]*c_1.dag()*vi*vi.dag()*c_1*vk*vk.dag()
                
                op3=epsilon*epsilon*constant11[i,k]*c_1.dag()
                op4=vi*vi.dag()*c_1*vk*vk.dag()
                op5=epsilon*epsilon*constant12[i,k]*c_1.dag()
                
                
                L_red=L_red+spre(-op2-op1.dag())+spost(-op1-op2.dag())
                L_red=L_red+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                
                op1=epsilon*epsilon*constant21[i,k]*vi*vi.dag()*c_N*vk*vk.dag()*c_N.dag()
                op2=epsilon*epsilon*constant22[i,k]*c_N.dag()*vi*vi.dag()*c_N*vk*vk.dag()
                
                op3=epsilon*epsilon*constant21[i,k]*c_N.dag()
                op4=vi*vi.dag()*c_N*vk*vk.dag()
                op5=epsilon*epsilon*constant22[i,k]*c_N.dag()
                
                
                L_red=L_red+spre(-op2-op1.dag())+spost(-op1-op2.dag())
                L_red=L_red+spre(op3)*spost(op4)+spre(op4)*spost(op5)+spre(op4.dag())*spost(op3.dag()) +spre(op5.dag())*spost(op4.dag())
                
                
                
        #Variables needed for for iterative-lgmres to work. 
        return_info=True
        #print('Redfield Liouvillian constructed, Computing steady-state ...')
        ss_redfield = steadystate(L_red,return_info=return_info)
        L_eigen = L_red.eigenenergies()
        print("Smallest eigenvalues of L_red are ", L_eigen[-3:])
        list_red.append(ss_redfield)

################################## Local Lindblad stuff ########################################
        gammah_plus =  gamma1*nbar(1,beta2,mu)
        gammah_minus = gamma1*(nbar(1,beta2,mu)+1)
        gammac_plus = gamma2*nbar(1+e,beta1,mu)  #beta1 is fixed at 1/0.1
        gammac_minus = gamma2*(nbar(1+e,beta1,mu)+1)

        L_lle = spre(-1.0j*(H_S)) + spost(1.0j*(H_S))
        op1 = create_sm(N,1).dag()
        op2 = create_sm(N,N).dag()
        L_lle = L_lle + gammah_plus*(spre(op1)*spost(op1.dag()) - 0.5*spre(op1.dag()*op1) - 0.5*spost(op1.dag()*op1)) + gammah_minus*(spre(op1.dag())*spost(op1) - 0.5*spre(op1*op1.dag()) - 0.5*spost(op1*op1.dag())) + gammac_plus*(spre(op2)*spost(op2.dag()) - 0.5*spre(op2.dag()*op2) - 0.5*spost(op2.dag()*op2)) + gammac_minus*(spre(op2.dag())*spost(op2) - 0.5*spre(op2*op2.dag()) - 0.5*spost(op2*op2.dag()))

        ss_lindblad2 = steadystate(L_lle)
        L_eigen = L_lle.eigenenergies()
        print("Smallest eigenvalues of L_lle are ", L_eigen[-3:])

        list_lle.append(ss_lindblad2)

        ########################## Another Local Lindblad version, with bath and spectral functions involved ###############################

        Delta1=(-1.0*epsilon*epsilon/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma1),weight='cauchy',wvar=w0list[0])[0] #Delta
        Deltadash1=(-1.0*epsilon*epsilon/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta1,mu1,gamma1),weight='cauchy',wvar=w0list[0])[0] #Delta
        
        
        DeltaN=(-1.0*epsilon*epsilon/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma2),weight='cauchy',wvar=w0list[N-1])[0] #Delta
        DeltadashN=(-1.0*epsilon*epsilon/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta2,mu2,gamma2),weight='cauchy',wvar=w0list[N-1])[0] #Delta

        H=H_S  +(Deltadash1+0.5*Delta1)*create_sigmaz(N,1)+(DeltadashN+0.5*DeltaN)*create_sigmaz(N,N) 

        L_lle = spre(-1.0j*(H)) + spost(1.0j*(H))
        op1 = create_sm(N,1).dag()
        op2 = create_sm(N,N).dag()
        L_lle = L_lle + gammah_plus*(spectral_bath(1,tb,1))*(spre(op1)*spost(op1.dag()) - 0.5*spre(op1.dag()*op1) - 0.5*spost(op1.dag()*op1)) + gammah_minus*(spectral_bath(1,tb,1))*(spre(op1.dag())*spost(op1) - 0.5*spre(op1*op1.dag()) - 0.5*spost(op1*op1.dag())) + gammac_plus*(spectral_bath(1+e,tb,1))*(spre(op2)*spost(op2.dag()) - 0.5*spre(op2.dag()*op2) - 0.5*spost(op2.dag()*op2)) + gammac_minus*(spectral_bath(1+e,tb,1))*(spre(op2.dag())*spost(op2) - 0.5*spre(op2*op2.dag()) - 0.5*spost(op2*op2.dag()))

        ss_lindblad2 = steadystate(L_lle)
        L_eigen = L_lle.eigenenergies()
        print("Smallest eigenvalues of L_lle2 are ", L_eigen[-3:])

        list_lle2.append(ss_lindblad2)
    redfield_ss.append(list_red)
    lle_ss.append(list_lle)
    lle_ss2.append(list_lle2)
    ele_ss.append(list_ele)"""

3 1
Integral calculations at beta2 = 10.0 and e = 0.01 are : 
Absolute frequency  for i = 0, k = 0 is  0.0
Absolute frequency  for i = 0, k = 1 is  0.9999009804864072
Absolute frequency  for i = 0, k = 2 is  1.0100990195135928
Absolute frequency  for i = 0, k = 3 is  2.01
Absolute frequency  for i = 1, k = 0 is  0.9999009804864072
Absolute frequency  for i = 1, k = 1 is  0.0
Absolute frequency  for i = 1, k = 2 is  0.010198039027185635
Absolute frequency  for i = 1, k = 3 is  1.0100990195135926
Absolute frequency  for i = 2, k = 0 is  1.0100990195135928
Absolute frequency  for i = 2, k = 1 is  0.010198039027185635
Absolute frequency  for i = 2, k = 2 is  0.0
Absolute frequency  for i = 2, k = 3 is  0.999900980486407
Absolute frequency  for i = 3, k = 0 is  2.01
Absolute frequency  for i = 3, k = 1 is  1.0100990195135926
Absolute frequency  for i = 3, k = 2 is  0.999900980486407
Absolute frequency  for i = 3, k = 3 is  0.0
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj

/Users/soumyadeepsarma/Desktop/Projects Soumyadeep/ICTS/SearchingLindbladians/python/helper_code_qutip.py:205: RuntimeWarning: overflow encountered in exp
  return 1/(np.exp(beta*(omega-mu))-1)
/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_89955/2862664082.py:173: RuntimeWarning: invalid value encountered in scalar multiply
  op1=epsilon*epsilon*constant11[i,k]*vi*vi.dag()*c_1*vk*vk.dag()*vm*vm.dag()*c_1.dag()*vj*vj.dag()
/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_89955/2862664082.py:174: RuntimeWarning: invalid value encountered in scalar multiply
  op2=epsilon*epsilon*constant12[i,k]*vm*vm.dag()*c_1.dag()*vj*vj.dag()*vi*vi.dag()*c_1*vk*vk.dag()
/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_89955/2862664082.py:176: RuntimeWarning: invalid value encountered in scalar multiply
  op3=epsilon*epsilon*constant11[i,k]*vm*vm.dag()*c_1.dag()*vj*vj.dag()
/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_89955/2862664082.py:178: RuntimeWarning:

Absolute frequency  for i = 0, k = 2 is  1.0908068168898195
Absolute frequency  for i = 0, k = 3 is  2.01
Absolute frequency  for i = 1, k = 0 is  0.9191931831101807
Absolute frequency  for i = 1, k = 1 is  0.0
Absolute frequency  for i = 1, k = 2 is  0.17161363377963879
Absolute frequency  for i = 1, k = 3 is  1.090806816889819
Absolute frequency  for i = 2, k = 0 is  1.0908068168898195
Absolute frequency  for i = 2, k = 1 is  0.17161363377963879
Absolute frequency  for i = 2, k = 2 is  0.0
Absolute frequency  for i = 2, k = 3 is  0.9191931831101803
Absolute frequency  for i = 3, k = 0 is  2.01
Absolute frequency  for i = 3, k = 1 is  1.090806816889819
Absolute frequency  for i = 3, k = 2 is  0.9191931831101803
Absolute frequency  for i = 3, k = 3 is  0.0
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (nan-infj)
(nan-infj) (n